In [1]:
import pandas as pd
import numpy as np
import config
from sqlalchemy import text
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)
pd.options.mode.copy_on_write = True

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [2]:
df = pd.read_csv("../data/ps_libreacces/PS_LibreAcces_Personne_activite_202303000000.txt", delimiter="|", low_memory=False, dtype={'Identifiant PP': 'str'})
df["inpp"]=df["Identification nationale PP"]
df["inpp"].nunique()

1615783

In [8]:
# df = df[df["Code postal (coord. structure)"].isna()==False]
# df["Identifiant PP"].nunique()
# df.columns
df=df[df["Code postal (coord. structure)"].notna()]
df["inpp"].nunique()


1089938

In [11]:
df=df[df["Code mode exercice"]=="L"]
df["inpp"].nunique()

412818

In [12]:
df[df["Code profession"]==60]["inpp"].nunique() #ameli 99467

101469

In [5]:
#df["Libellé savoir-faire"].unique()

In [6]:
df2=df[df["Libellé savoir-faire"]=="Pédiatrie"]
df2["inpp"].nunique() #2678

2678

In [7]:
sql = """
select * from specialite s
join specialite_profession sp on sp.specialite_id=s.id
join profession_diplome pd on pd.profession_id=sp.profession_id
join diplome d on d.id=pd.diplome_id
where s.id=7
"""
spe=pd.read_sql(sql, config.connection_string)
spe

,id,label,label_long,psp_spe_snds,profession_id,specialite_id,diplome_id,code_type_diplome,libelle_type_diplome,code_diplome,libelle_diplome,is_savoir_faire
0,554,Pediatre,Pédiatre,12,60,7,554,S,Spécialité ordinale,SM40,Pédiatrie,True
1,43,Pediatre,Pédiatre,12,60,7,43,CES,Certficat d'études spéciales,CESM16,CES Pédiatrie,False
2,164,Pediatre,Pédiatre,12,60,7,164,DEES,Diplôme Européen d'Etudes Spécialisées,DIP155,DEES Pédiatrie,False
3,87,Pediatre,Pédiatre,12,60,7,87,DES,Diplôme d'Etudes Spécialisées,DSM39,DES Pédiatrie,False
4,777,Pediatre,Pédiatre,12,60,7,777,DES,Diplôme d'Etudes Spécialisées,DSM89,DES Pédiatrie option néonatologie,False
5,781,Pediatre,Pédiatre,12,60,7,781,DES,Diplôme d'Etudes Spécialisées,DSM90,DES Pédiatrie option neuropédiatrie,False
6,774,Pediatre,Pédiatre,12,60,7,774,DES,Diplôme d'Etudes Spécialisées,DSM91,DES Pédiatrie option pneumopédiatrie,False
7,782,Pediatre,Pédiatre,12,60,7,782,DES,Diplôme d'Etudes Spécialisées,DSM92,DES Pédiatrie option réanimation pédiatrique,False


In [8]:
# df.columns
df["code_diplome"]=df["Code savoir-faire"]
df3 = spe.merge(df, on="code_diplome", how="left", suffixes=('', ''))
df3["inpp"].nunique()

2678

In [9]:
sql = """
select distinct(pa.inpp) inpp from personne_activite pa
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id
join adresse_norm an on an.id=pands.adresse_norm_id
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on d.id=pad.diplome_id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on p.id=pd.profession_id
join specialite_profession sp on sp.profession_id=p.id
where pands.date_source_id=2303
and pands.code_mode_exercice='L'
and sp.specialite_id=7
and pands.adresse_norm_id is not null
order by pa.inpp
"""
sqldf=pd.read_sql(sql, config.connection_string)
sqldf["inpp"].nunique()

2595

In [10]:
m = sqldf.merge(df3, on="inpp", how="left", suffixes=('', ''))
m["inpp"].nunique()

2595

In [11]:
errors=m[m["id"].isna()]
errors

,inpp,id,label,label_long,psp_spe_snds,profession_id,specialite_id,diplome_id,code_type_diplome,libelle_type_diplome,code_diplome,libelle_diplome,is_savoir_faire,Type d'identifiant PP,Identifiant PP,Identification nationale PP,Code civilité d'exercice,Libellé civilité d'exercice,Code civilité,Libellé civilité,Nom d'exercice,Prénom d'exercice,Code profession,Libellé profession,Code catégorie professionnelle,Libellé catégorie professionnelle,Code type savoir-faire,Libellé type savoir-faire,Code savoir-faire,Libellé savoir-faire,Code mode exercice,Libellé mode exercice,Numéro SIRET site,Numéro SIREN site,Numéro FINESS site,Numéro FINESS établissement juridique,Identifiant technique de la structure,Raison sociale site,Enseigne commerciale site,Complément destinataire (coord. structure),Complément point géographique (coord. structure),Numéro Voie (coord. structure),Indice répétition voie (coord. structure),Code type de voie (coord. structure),Libellé type de voie (coord. structure),Libellé Voie (coord. structure),Mention distribution (coord. structure),Bureau cedex (coord. structure),Code postal (coord. structure),Code commune (coord. structure),Libellé commune (coord. structure),Code pays (coord. structure),Libellé pays (coord. structure),Téléphone (coord. structure),Téléphone 2 (coord. structure),Télécopie (coord. structure),Adresse e-mail (coord. structure),Code Département (structure),Libellé Département (structure),Ancien identifiant de la structure,Autorité d'enregistrement,Code secteur d'activité,Libellé secteur d'activité,Code section tableau pharmaciens,Libellé section tableau pharmaciens,Code rôle,Libellé rôle,Code genre activité,Libellé genre activité,Unnamed: 56
965,810002728797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2348,810100518108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2674,810100838175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
errors["inpp"].nunique()

3

In [33]:
sql = """
select ps.* from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on d.code_diplome=ps.code_diplome
--join diplome d on d.id=pad.diplome_id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on p.id=pd.profession_id
join specialite_profession sp on sp.profession_id=p.id
join adresse_norm an on an.id=pands.adresse_norm_id
where ps.date_source_id=2303
and ps.code_mode_exercice='L'
and sp.specialite_id=7
and pands.adresse_norm_id is not null
order by ps.inpp
"""
bd=pd.read_sql(sql, config.connection_string)
bd["inpp"].nunique()


2592

In [14]:
bd[bd["inpp"]=="810002728797"]

,inpp,code_mode_exercice,cp,code_diplome,date_source_id


In [15]:
m2 = bd.merge(df3, on="inpp", how="left", suffixes=('left', ''))
m2["inpp"].nunique()

2592

In [17]:
errors2=m2[m2["id"].isna()]
errors2

,inpp,code_mode_exercice,cp,code_diplomeleft,date_source_id,id,label,label_long,psp_spe_snds,profession_id,specialite_id,diplome_id,code_type_diplome,libelle_type_diplome,code_diplome,libelle_diplome,is_savoir_faire,Type d'identifiant PP,Identifiant PP,Identification nationale PP,Code civilité d'exercice,Libellé civilité d'exercice,Code civilité,Libellé civilité,Nom d'exercice,Prénom d'exercice,Code profession,Libellé profession,Code catégorie professionnelle,Libellé catégorie professionnelle,Code type savoir-faire,Libellé type savoir-faire,Code savoir-faire,Libellé savoir-faire,Code mode exercice,Libellé mode exercice,Numéro SIRET site,Numéro SIREN site,Numéro FINESS site,Numéro FINESS établissement juridique,Identifiant technique de la structure,Raison sociale site,Enseigne commerciale site,Complément destinataire (coord. structure),Complément point géographique (coord. structure),Numéro Voie (coord. structure),Indice répétition voie (coord. structure),Code type de voie (coord. structure),Libellé type de voie (coord. structure),Libellé Voie (coord. structure),Mention distribution (coord. structure),Bureau cedex (coord. structure),Code postal (coord. structure),Code commune (coord. structure),Libellé commune (coord. structure),Code pays (coord. structure),Libellé pays (coord. structure),Téléphone (coord. structure),Téléphone 2 (coord. structure),Télécopie (coord. structure),Adresse e-mail (coord. structure),Code Département (structure),Libellé Département (structure),Ancien identifiant de la structure,Autorité d'enregistrement,Code secteur d'activité,Libellé secteur d'activité,Code section tableau pharmaciens,Libellé section tableau pharmaciens,Code rôle,Libellé rôle,Code genre activité,Libellé genre activité,Unnamed: 56


In [36]:
sql = """
select ps2.* from apl.ps_libreacces ps2
join personne_activite pa on pa.inpp=ps2.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps2.date_source_id
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on d.code_diplome=ps2.code_diplome
--join diplome d on d.id=pad.diplome_id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on p.id=pd.profession_id
join specialite_profession sp on sp.profession_id=p.id
join adresse_norm an on an.id=pands.adresse_norm_id
left outer join ps on ps.key=pa.inpp
--join tarif t on t.ps_id=ps.id
--join tarif_date_source tds on tds.tarif_id=t.id and tds.date_source_id=ps2.date_source_id
--join cabinet c on t.cabinet_id=c.id
--join adresse_raw ar on c.adresse_raw_id=ar.id
--join adresse_norm an2 on ar.adresse_norm_id=an2.id
where ps2.date_source_id=2303
and ps2.code_mode_exercice='L'
and sp.specialite_id=7
and pands.adresse_norm_id is not null
order by ps2.inpp
"""
bdps=pd.read_sql(sql, config.connection_string)
bdps["inpp"].nunique()


0